In [1]:
import DataBroker
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.autograd import Variable

In [74]:
broker = DataBroker.DataBroker()
broker_data = broker.get_all_company_data('CHR.CO')
broker_data_np = broker_data.values[:, :4]

device = torch.device('cuda')
epochs = 1000
batchsize = 128
learning_rate = 1e-4

dataset = torch.from_numpy(broker_data_np)
dataloader = DataLoader(dataset, batch_size=batchsize, shuffle=True)

dict_keys(['PNDORA.CO', 'VWS.CO', 'DANSKE.CO', 'CHR.CO', 'GOOGL', 'NFLX', 'TSLA'])


In [75]:
broker_data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-06-03,94.000000,96.800003,93.599998,95.000000,83.481415,6624457
2010-06-04,96.000000,98.250000,95.500000,96.750000,85.019241,2321487
2010-06-07,95.500000,100.300003,94.500000,99.900002,87.787292,1117218
2010-06-08,101.000000,102.500000,98.199997,98.750000,86.776741,696090
2010-06-09,99.199997,99.199997,96.500000,97.800003,85.941917,574212


In [77]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(4, 1),
        )
        self.decoder = nn.Sequential(
            nn.Linear(1, 4),
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [78]:
model = autoencoder().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-6)

In [79]:
for epoch in range(epochs):
    for data in dataloader:
        model_in = Variable(data).to(device).float()
        #### Forward
        output = model(model_in)
        loss = criterion(output, model_in)
        #### Backward
        loss.backward()
        optimizer.step()
    if (epoch + 1) % 50 == 0:
        print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, epochs, loss.item()))

epoch [50/1000], loss:24849.7520
epoch [100/1000], loss:23781.9863
epoch [150/1000], loss:13480.3252
epoch [200/1000], loss:9417.0781
epoch [250/1000], loss:3358.9873
epoch [300/1000], loss:228.5078
epoch [350/1000], loss:2856.3789
epoch [400/1000], loss:5867.1875
epoch [450/1000], loss:5722.8423
epoch [500/1000], loss:3002.6672
epoch [550/1000], loss:1749.2642
epoch [600/1000], loss:1409.5004
epoch [650/1000], loss:1059.2450
epoch [700/1000], loss:1773.2827
epoch [750/1000], loss:657.2342
epoch [800/1000], loss:635.6194
epoch [850/1000], loss:795.6905
epoch [900/1000], loss:1041.7300
epoch [950/1000], loss:410.9237
epoch [1000/1000], loss:487.5984


In [80]:
model_in = Variable(torch.from_numpy(broker_data_np[:4])).to(device).float()
print(model_in)
print(model(model_in))
model.encoder(model_in)

tensor([[ 94.0000,  96.8000,  93.6000,  95.0000],
        [ 96.0000,  98.2500,  95.5000,  96.7500],
        [ 95.5000, 100.3000,  94.5000,  99.9000],
        [101.0000, 102.5000,  98.2000,  98.7500]], device='cuda:0')
tensor([[ 90.6514,  97.0587,  85.0059,  91.5811],
        [ 92.1384,  98.6356,  86.3942,  93.0553],
        [ 95.0561, 101.7299,  89.1186,  95.9482],
        [ 95.5167, 102.2184,  89.5486,  96.4048]],
       device='cuda:0', grad_fn=<ThAddmmBackward>)


tensor([[57.3566],
        [58.2963],
        [60.1404],
        [60.4315]], device='cuda:0', grad_fn=<ThAddmmBackward>)